## This is currently not working

https://medium.com/@pallawi.ds/semantic-segmentation-with-u-net-train-and-test-on-your-custom-data-in-keras-39e4f972ec89

In [88]:
# Install the plaidml backend
import plaidml.keras
plaidml.keras.install_backend()

# Help MacOS be able to use Keras
import os
os.environ["KERAS_BACKEND"] = "plaidml.keras.backend"
os.environ['KMP_DUPLICATE_LIB_OK']='True'

In [89]:
# better progress bars for jupyter and won't kill model fit
# Command line the following line to get tqdm installed properly
# python -m pip install tqdm
from keras_tqdm import TQDMNotebookCallback
# keras, model definition...
cb = TQDMNotebookCallback()
setattr(cb,'on_train_batch_begin',lambda x,y:None)
setattr(cb,'on_train_batch_end',lambda x,y:None)

In [442]:

import numpy as np
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from keras.preprocessing.image import img_to_array
from keras.datasets import mnist
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Flatten, Activation, Input
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K
from keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
import pandas as pd
import time

In [475]:
batch_size = 32
epochs = 5
# input image dimensions
img_rows, img_cols = 256, 256
input_shape = (img_rows, img_cols, 3)

### Images to Arrays

In [462]:
images = glob.glob('../images/network_training/build/0/*.tif')
img_list = []
for item in images[:10]:
    
    item = load_img(item)
    item = cv2.resize(image, 256, 256)
    img_list.append(item)

In [463]:
len(img_list)

180

### Masks to Arrays

In [465]:
masks = glob.glob('../images/training/build_mask/0/*.tif')
mask_list = []
for item in masks:
    
    item = load_img(item)
    item = cv2.resize(image, 256, 256)
    mask_list.append(item)

In [466]:
len(mask_list)

180

In [ ]:
data = np.array(img_list, dtype='float') / 255

In [429]:
K.image_data_format()

'channels_last'

In [468]:
X_train, X_test, y_train, y_test = train_test_split(img_list, mask_list, test_size=.2, random_state=77)

### Data Generator For Images and Masks

In [471]:
#start timer
start_time = time.time()

# we create two instances with the same arguments
data_gen_args = dict(featurewise_center=True,
                     featurewise_std_normalization=True,
                     rotation_range=90,
                     width_shift_range=0.1,
                     height_shift_range=0.1,
                     zoom_range=0.2,
                     data_format='channels_last')
image_datagen = ImageDataGenerator(**data_gen_args)
mask_datagen = ImageDataGenerator(**data_gen_args)

# Provide the same seed and keyword arguments to the fit and flow methods
seed = 77
image_datagen.fit(test_image, augment=True, seed=seed)
mask_datagen.fit(test_mask, augment=True, seed=seed)

image_generator = image_datagen.flow(
    X_train,
#     class_mode=None,
    seed=seed)

mask_generator = mask_datagen.flow(
    y_train,
#     class_mode=None,
    seed=seed)

# combine generators into one which yields image and masks
train_generator = zip(image_generator, mask_generator)


#End Time
elapsed_time = time.time() - start_time
elapsed_min = elapsed_time / 60
print(f'Total Time: {round(elapsed_min, 3)} minutes.')

Total Time: 0.001 minutes.


In [481]:


# my_gen = ImageDataGenerator(featurewise_center=True,
#                          featurewise_std_normalization=True,
#                          rotation_range=90,
#                          width_shift_range=0.1,
#                          height_shift_range=0.1,
#                          zoom_range=0.2,
#                          data_format='channels_last')




# # Fit model
# model.fit_generator(my_gen.flow(X_train, y_train, batch_size=batch_size),
#            epochs=epochs,
#              # steps per epoch should be total samples / batch size
#             steps_per_epoch=len(X_train)//batch_size,
#               verbose=0)
#            callbacks=[cb])   
#           validation_data=(x_test, y_test)   -- This goes inside the fit
# score = model.evaluate(x_test, y_test, verbose=1)
# print('Test loss:', score[0])
# print('Test accuracy:', score[1])

ValueError: Error when checking model input: the list of Numpy arrays that you are passing to your model is not the size the model expected. Expected to see 1 array(s), but instead got the following list of 144 arrays: [array([[[[0.54509807, 0.50980395, 0.4509804 ],
         [0.4       , 0.39607844, 0.36078432],
         [0.27058825, 0.32156864, 0.30980393],
         ...,
         [0.3254902 , 0.3647059 , 0.33333334...

In [457]:
model.history

In [458]:
len(X_train)

5